<a href="https://colab.research.google.com/github/yk-Jeong/dsnote/blob/main/9%EC%9E%A5_RandomForest_%EC%A4%91%EA%B3%A0%EC%B0%A8%EA%B0%80%EA%B2%A9%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. lib and data load

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [69]:
data = 'https://media.githubusercontent.com/media/musthave-ML10/data_source/main/car.csv'
df = pd.read_csv(data)

In [70]:
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


mileage, engine, max_power -> 단위 제거해 수치형으로

torque 분할 

fuel, seller_type, transmission -> 원핫인코딩

owner -> 단계별 변수로 

In [72]:
# 이상치 확인 가능 

round(df.describe(), 2)

,year,selling_price,km_driven,seats
count,8128.00,8128.00,8128.00,7907.00
mean,2013.80,638271.81,69819.51,5.42
std,4.04,806253.40,56550.55,0.96
min,1983.00,29999.00,1.00,2.00
25%,2011.00,254999.00,35000.00,5.00
50%,2015.00,450000.00,60000.00,5.00
75%,2017.00,675000.00,98000.00,5.00
max,2020.00,10000000.00,2360457.00,14.00


### 1. EDA


In [73]:
# engine

df['engine'].str.split(expand=True)

,0,1
0,1248,CC
1,1498,CC
2,1497,CC
3,1396,CC
4,1298,CC
...,...,...
8123,1197,CC
8124,1493,CC
8125,1248,CC
8126,1396,CC


In [74]:
# 단위는 모두 CC로 일치하는 것을 알 수 있음 

df['engine'].str.split(expand=True)[1].unique()

array(['CC', nan], dtype=object)

In [75]:
# 문자 표기 삭제 후 숫자형으로 변경, 컬럼명에 단위명을 표기 

df['engine'] = df['engine'].str.split(expand=True)[0].astype('float32')
df.rename(columns = {'engine':'engine_cc'}, inplace=True)

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine_cc      7907 non-null   float32
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float32(1), float64(1), int64(3), object(8)
memory usage: 793.9+ KB


max_power_bhp 항목에서 에러 발생해 해결 

In [77]:
df[df['max_power'] == ' bhp']

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine_cc,max_power,torque,seats
4933,Maruti Omni CNG,2000,80000,100000,CNG,Individual,Manual,Second Owner,10.9 km/kg,796.0,bhp,NaN,8.0


In [78]:
def isFloat(value):
  try:
    num = float(value)
    return num
  except ValueError:
    return np.NaN # numpy null 값으로 적용

In [79]:
# applpy 함수를 활용해 df에 적용하고 결측치 삭제 

df['max_power'] = df['max_power'].apply(isFloat)
df.drop('max_power', axis=1, inplace=True)

mileage의 경우, 별도의 단위가 존재 -> fuel 변수를 활용해 단위 통일

In [80]:
df['mileage'].str.split(expand=True)[1].unique()

array(['kmpl', 'km/kg', nan], dtype=object)

In [81]:
df['fuel'].unique()

array(['Diesel', 'Petrol', 'LPG', 'CNG'], dtype=object)

In [82]:
def mile(df):
  if df['fuel'] == 'Diesel':
    return df['mileage'] / 73.56
  elif df['fuel'] == 'Petrol':
    return df['mileage'] / 80.43
  elif df['fuel'] == 'LPG':
    return df['mileage'] / 40.85
  else:
    return df['mileage'] / 44.23

In [83]:
# 수치형으로 수정 후, 리터당 가격으로 변경

df['mileage'] = df['mileage'].str.split(expand=True)[0].astype('float32')
df['mileage'] = df.apply(mile, axis=1)

torque 

In [84]:
df['torque'].head()

0              190Nm@ 2000rpm
1         250Nm@ 1500-2500rpm
2       12.7@ 2,700(kgm@ rpm)
3    22.4 kgm at 1750-2750rpm
4       11.5@ 4,500(kgm@ rpm)
Name: torque, dtype: object

In [85]:
df['torque'] = df['torque'].str.lower()

In [86]:
# 단위 변환을 위한 함수 선언 후 적용

def torque_unit(x):
  if 'nm' in str(x):
    return 'Nm'
  elif 'kgm'in str(x):
    return 'kgm'

df['torque_unit'] = df['torque'].apply(torque_unit)

In [87]:
df['torque_unit'].unique()

array(['Nm', 'kgm', None], dtype=object)

In [88]:
# Nm, kgm 이외의 단위가 존재하는지 확인 

df[df['torque_unit'].isna()]['torque'].unique()

array([nan, '250@ 1250-5000rpm', '510@ 1600-2400', '110(11.2)@ 4800',
       '210 / 1900'], dtype=object)

In [89]:
df['torque_unit'].unique()

array(['Nm', 'kgm', None], dtype=object)

In [90]:
# torque_unit이 None인 경우의 torque 값 확인

df[df['torque_unit'].isna()]['torque'].unique()

array([nan, '250@ 1250-5000rpm', '510@ 1600-2400', '110(11.2)@ 4800',
       '210 / 1900'], dtype=object)

In [91]:
df['torque_unit'].fillna('Nm', inplace=True)

In [92]:
# 문자열 분리 함수 정의

def split_num(x):
  x = str(x)
  for i, j in enumerate(x):
    if j not in '0123456789.':
      cut = i
      break
  return x[:cut]

In [93]:
df['torque'] = df['torque'].apply(split_num)
df['torque']

0         190
1         250
2        12.7
3        22.4
4        11.5
        ...  
8123    113.7
8124       24
8125      190
8126      140
8127      140
Name: torque, Length: 8128, dtype: object

In [96]:
# 비어 있는 string이 있어 float으로 형변환되지 않으므로 공백문자를 결측치로 대체한 뒤 float으로 형변환

df['torque'] = df['torque'].replace('', np.NaN)
df['torque'] = df['torque'].astype('float32')

In [98]:
df['torque'].head()

0    190.0
1    250.0
2     12.7
3     22.4
4     11.5
Name: torque, dtype: float32

In [99]:
# 단위 일치를 위한 함수 정의 

def torque_trans(x):
  if x['torque_unit'] == 'kgm':
    return x['torque'] * 9.8066
  else:
    return x['torque']

In [100]:
# 형변환 적용 후 unit 열 삭제 

df['torque'] = df.apply(torque_trans, axis=1)
df.drop('torque_unit', axis=1, inplace=True)

#### 이름(name) 피처 핸들링

In [102]:
# 확인: 가장 앞쪽에 브랜드가 나오는 것을 알 수 있음

df['name'].head()

0          Maruti Swift Dzire VDI
1    Skoda Rapid 1.5 TDI Ambition
2        Honda City 2017-2020 EXi
3       Hyundai i20 Sportz Diesel
4          Maruti Swift VXI BSIII
Name: name, dtype: object

In [104]:
# 브랜드만 남기고 삭제 

df['name'] = df['name'].str.split(expand=True)[0] 

In [105]:
df['name'].unique()

array(['Maruti', 'Skoda', 'Honda', 'Hyundai', 'Toyota', 'Ford', 'Renault',
       'Mahindra', 'Tata', 'Chevrolet', 'Fiat', 'Datsun', 'Jeep',
       'Mercedes-Benz', 'Mitsubishi', 'Audi', 'Volkswagen', 'BMW',
       'Nissan', 'Lexus', 'Jaguar', 'Land', 'MG', 'Volvo', 'Daewoo',
       'Kia', 'Force', 'Ambassador', 'Ashok', 'Isuzu', 'Opel', 'Peugeot'],
      dtype=object)

In [106]:
# Land를 Landrover로 변경

df['name'] = df['name'].replace('Land', 'Landrover')

#### 결측치 처리

In [110]:
df.isna().mean() # 결측치 비율 확인

name             0.000000
year             0.000000
selling_price    0.000000
km_driven        0.000000
fuel             0.000000
seller_type      0.000000
transmission     0.000000
owner            0.000000
mileage          0.027190
engine_cc        0.027190
torque           0.027313
seats            0.027190
dtype: float64

In [112]:
# 2~3% 내외 적은 비율이므로 모두 삭제 

df.dropna(inplace=True)

#### 텍스트 컬럼 더미 변환 

In [113]:
df = pd.get_dummies(df, columns = ['name', 'fuel', 'seller_type', 'transmission', 'owner'])
df.head()

### 2. Modeling

In [115]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(df.drop('selling_price', axis=1), df['selling_price'], test_size=0.2, random_state=100)

In [119]:
# 예시로 하나만 확인 

X_train.head()

,year,km_driven,mileage,engine_cc,torque,seats,name_Ambassador,name_Ashok,name_Audi,name_BMW,...,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Automatic,transmission_Manual,owner_First Owner,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
2270,2016,70000,0.234987,1197.0,114.00,5.0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
3817,2013,80000,0.241204,1198.0,109.00,5.0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
3533,2016,11000,0.177794,1598.0,153.00,5.0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,0
1766,2019,90000,0.250404,1197.0,113.75,5.0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
1869,2017,25272,0.261221,1197.0,113.00,5.0,0,0,0,0,...,0,0,1,0,1,1,0,0,0,0


In [131]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train) # 학습
train_pred = model.predict(X_train) # 예측
test_pred = model.predict(X_test)

#### 평가 

In [132]:
from sklearn.metrics import mean_squared_error

print('train_rmse:', round(mean_squared_error(y_train, train_pred) ** 0.5, 3))
print('test_rmse:', round(mean_squared_error(y_test, test_pred) ** 0.5, 3))

train_rmse: 59365.327
test_rmse: 133513.845


### K-fold cross validation

In [133]:
from sklearn.model_selection import KFold

In [138]:
# 인덱스 정리

df.reset_index(drop=True, inplace=True)
df

,year,selling_price,km_driven,mileage,engine_cc,torque,seats,name_Ambassador,name_Ashok,name_Audi,...,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Automatic,transmission_Manual,owner_First Owner,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,190.000000,5.0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
1,2014,370000,120000,0.287384,1498.0,250.000000,5.0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
2,2006,158000,140000,0.220067,1497.0,124.543818,5.0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
3,2010,225000,127000,0.312670,1396.0,219.667836,5.0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
4,2007,130000,120000,0.200174,1298.0,112.775900,5.0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7901,2013,320000,110000,0.230014,1197.0,113.699997,5.0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
7902,2007,135000,119000,0.228385,1493.0,235.358400,5.0,0,0,0,...,0,1,0,0,1,0,1,0,0,0
7903,2009,382000,120000,0.262371,1248.0,190.000000,5.0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
7904,2013,290000,25000,0.320419,1396.0,140.000000,5.0,0,0,0,...,0,1,0,0,1,1,0,0,0,0


In [140]:
kf = KFold(n_splits=5) # KFold 객체 생성

In [144]:
# 독립/종속변수 설정

X = df.drop('selling_price', axis=1)
y = df['selling_price']

In [145]:
for i, j in kf.split(X):
  print(i, j) # 훈련 세트-시험 세트 순서로 각각 5개 행의 인덱스를 출력 

[1582 1583 1584 ... 7903 7904 7905] [   0    1    2 ... 1579 1580 1581]
[   0    1    2 ... 7903 7904 7905] [1582 1583 1584 ... 3160 3161 3162]
[   0    1    2 ... 7903 7904 7905] [3163 3164 3165 ... 4741 4742 4743]
[   0    1    2 ... 7903 7904 7905] [4744 4745 4746 ... 6322 6323 6324]
[   0    1    2 ... 6322 6323 6324] [6325 6326 6327 ... 7903 7904 7905]


In [166]:
# 각 값들을 저장할 빈 리스트 생성 

train_rmse_total = []
test_rmse_total = []

for train_index, test_index in kf.split(X):
  
  # X_train, X_test 정의
  X_train, X_test = X.loc[train_index], X.loc[test_index]

  # y_train, y_test 정의
  y_train, y_test = y[train_index], y[test_index]

  # 모델 정의(하이퍼파라미터 포함)
  # n_estimators 결정 트리의 수, max_depth 최대 깊이 제한, min_samples_split 노드 분리의 기준이 될 데이터의 수, min_samples_leaf 분리된 노드에 있을 최소 데이터 수 
  model = RandomForestRegressor(n_estimators = 100, max_depth=50, min_samples_split=5, min_samples_leaf=1, n_jobs=-1, random_state=100)
  model.fit(X_train, y_train)

  # 적합
  train_pred = model.predict(X_train)
  test_pred = model.predict(X_test)

  # rmse 계산
  train_rmse = mean_squared_error(y_train, train_pred) ** 0.5
  test_rmse = mean_squared_error(y_test, test_pred) ** 0.5

  train_rmse_total.append(train_rmse)
  test_rmse_total.append(test_rmse)

In [167]:
print('train_rmse:', sum(train_rmse_total) / 5)
print('test_rmse:', sum(test_rmse_total) / 5)

train_rmse: 75276.28182039449
test_rmse: 154632.25404366056
